In [42]:
import pandas as pd
import numpy as np

In [43]:
수익성 = pd.read_csv('./datasets/코스피/코스피수익성.csv',encoding='cp949')
안정성 = pd.read_csv('./datasets/코스피/코스피 안정성.csv',encoding='cp949')
활동성 = pd.read_csv('./datasets/코스피/코스피활동성.csv',encoding='cp949')
성장성 = pd.read_csv('./datasets/코스피/코스피 성장성 계산용.csv',encoding='cp949')
현금흐름 = pd.read_csv('./datasets/코스피/코스피 현금흐름 + 누적수익성비율.csv',encoding='cp949')
연구개발 = pd.read_csv('./datasets/코스피/코스피 rs.csv',encoding='cp949')
EBIT = pd.read_csv('./datasets/코스피/에비~ 코스피.csv',encoding='cp949')
per_pbr_총부채 = pd.read_csv('./datasets/코스피/per pbr 총부채 코스피.csv',encoding='cp949')
직원수_업력 = pd.read_csv('./datasets/코스피/코스피 직원수 급여 업력 외국인 대주주.csv',encoding='cp949')
시가총액 = pd.read_csv('./datasets/코스피/코스피시가총액.csv',index_col=0)  #### 시총도 붙여야됨
단기차입금 = pd.read_csv('./datasets/코스피/코스피 단기차입금+유동성장기부채.csv',encoding='cp949')
거시변수 = pd.read_excel('./datasets/거시경제 변수/거시변수.xlsx')
상장일 = pd.read_csv('./datasets/코스피/코스피 상장일.csv',encoding='cp949')
유형자산 = pd.read_csv('./datasets/코스피/코스피 유형증가율.csv',encoding='cp949')


In [44]:
df = 안정성.merge(수익성,on=['회사명','거래소코드','회계년도'],how='inner')
df = df.merge(활동성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(성장성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(현금흐름,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(연구개발,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(EBIT,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(per_pbr_총부채,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(직원수_업력,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(단기차입금,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(상장일,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(유형자산,on=['회사명','거래소코드','회계년도'],how='left')

In [45]:
len(df.columns)

97

In [46]:
## 거래소코드 패딩
def padding(x):
    x = str(x)
    return x.rjust(6, "0")
## 외국계기업 제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

# 회계년도컬럼, 년도만 뽑아서 int화
def change_year(x):
    return int(x[:4])

def func_not(x):
    return not x

def change_year_2(x):
    return (x[:4])

In [47]:
## 거래소 코드 패딩
df['거래소코드'] = df['거래소코드'].map(padding)
# 회계년도 제거
df['회계년도'] = df['회계년도'].map(change_year)

In [48]:
df.isna().sum()

회사명                                 0
거래소코드                               0
회계년도                                0
자산(*)(요약)(IFRS)                   170
자본(*)(요약)(IFRS)                   170
                                 ... 
[제조]      단기차입금(요약)(IFRS연결)      1514
[제조]      유동성장기부채(요약)(IFRS연결)    1514
상장일                                 0
유형자산의 증가(*)(IFRS)(천원)             170
유형자산의 증가(*)(IFRS연결)(천원)          1514
Length: 97, dtype: int64

In [49]:
df[df.duplicated(subset=['거래소코드','회계년도'])]

,회사명,거래소코드,회계년도,자산(*)(요약)(IFRS),자본(*)(요약)(IFRS),차입금의존도(IFRS)(백만원),유보율(IFRS),현금비율(IFRS),부채비율(IFRS),당좌비율(IFRS),...,설립일,외국인_주식분포비율,보통주식비율,[제조] 단기차입금(요약)(IFRS),[제조] 유동성장기부채(요약)(IFRS),[제조] 단기차입금(요약)(IFRS연결),[제조] 유동성장기부채(요약)(IFRS연결),상장일,유형자산의 증가(*)(IFRS)(천원),유형자산의 증가(*)(IFRS연결)(천원)
7101,평산차업집단유한공사,950010,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2002/01/03,NaN,NaN,NaN,NaN,4115.0,0.0,2007/11/26,NaN,8660768.0


In [50]:
## 중복되는 데이터 제거
df.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
df.reset_index(drop=True, inplace=True)

## 없음


In [51]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 8010 개
전체 기업 수 797


In [52]:
## 외국계 기업 제거
print("제거할 데이터 수 : ", len(df[df["거래소코드"].map(func_isnt_fcompany).map(func_not)]))
print('제거할 외국계 기업 수',len(df[~df["거래소코드"].map(func_isnt_fcompany)].회사명.unique()))

df = df[df["거래소코드"].map(func_isnt_fcompany)]
print('남은 데이터 수',df.shape[0]) # 외국계기업 데이터 24개 제거
print('남은 기업 수',len(df['회사명'].unique()))

제거할 데이터 수 :  24
제거할 외국계 기업 수 5
남은 데이터 수 7986
남은 기업 수 792


In [53]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(df["회사명"].unique()))
print("거래소코드수 : ", len(df["거래소코드"].unique()))

회사수 :  792
거래소코드수 :  795


In [54]:
## 거래소 코드 중복기업 전처리
list_overlap_code = list()
for i in list(df["회사명"].unique()):
    if len(df[df["회사명"]==i]["거래소코드"].unique()) > 1:
        list_overlap_code.append(i)
print("거래소코드가 2개인 기업 수", len(list_overlap_code), "개")

거래소코드가 2개인 기업 수 3 개


In [55]:
list_overlap_code

['(주)신성이엔지', '삼성물산(주)', '에스케이(주)']

In [56]:
# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (df[df["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(df["회사명"].unique())


dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = df[df["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)

In [57]:
# 회사명이 2개인 거래소코드 존재 확인 
list_overlap_code = list()
for i in list(df["거래소코드"].unique()):
    if len(df[df["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

[]

In [58]:
## 연결 결측치 개별로 채우기
for i in range(df.shape[0]):
    if pd.isnull(df.loc[i,'유보율(IFRS연결)(백만원)']):
       df.loc[i,'유보율(IFRS연결)(백만원)'] = df.loc[i,'유보율(IFRS)']
    if pd.isnull(df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'자산(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'자산(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'자산(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'부채(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'부채(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'부채(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]자본(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]자본(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]자본(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'당기순이익(손실)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'당기순이익(손실)(요약)(IFRS연결)(백만원)'] = df.loc[i,'당기순이익(손실)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)'] = df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'금융원가(요약)(IFRS연결)(백만원)']):
       df.loc[i,'금융원가(요약)(IFRS연결)(백만원)'] = df.loc[i,'금융원가(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'금융수익(요약)(IFRS연결)(백만원)']):
       df.loc[i,'금융수익(요약)(IFRS연결)(백만원)'] = df.loc[i,'금융수익(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'매출액(수익)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'매출액(수익)(요약)(IFRS연결)(백만원)'] = df.loc[i,'매출액(수익)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'연구개발비(IFRS연결)(천원)']):
       df.loc[i,'연구개발비(IFRS연결)(천원)'] = df.loc[i,'연구개발비(IFRS)(천원)']
    if pd.isnull(df.loc[i,'[제조]      유형자산(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]      유형자산(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]      유형자산(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'자본금회전률(IFRS연결)']):
       df.loc[i,'자본금회전률(IFRS연결)'] = df.loc[i,'자본금회전률(IFRS)']
    if pd.isnull(df.loc[i,'총자본회전률(IFRS연결)']):
       df.loc[i,'총자본회전률(IFRS연결)'] = df.loc[i,'총자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'재고자산회전률(IFRS연결)']):
       df.loc[i,'재고자산회전률(IFRS연결)'] = df.loc[i,'재고자산회전률(IFRS)']
    if pd.isnull(df.loc[i,'유형자산회전율(IFRS연결)']):
       df.loc[i,'유형자산회전율(IFRS연결)'] = df.loc[i,'유형자산회전율(IFRS)']
    if pd.isnull(df.loc[i,'자기자본회전률(IFRS연결)']):
       df.loc[i,'자기자본회전률(IFRS연결)'] = df.loc[i,'자기자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'타인자본회전률(IFRS연결)']):
       df.loc[i,'타인자본회전률(IFRS연결)'] = df.loc[i,'타인자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'[제조]경영자본순이익률(IFRS연결)']):
       df.loc[i,'[제조]경영자본순이익률(IFRS연결)'] = df.loc[i,'[제조]경영자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'[제조]금융비용부담률(IFRS연결)']):
       df.loc[i,'[제조]금융비용부담률(IFRS연결)'] = df.loc[i,'[제조]금융비용부담률(IFRS)']
    if pd.isnull(df.loc[i,'[제조]총자본순이익률(IFRS연결)']):
       df.loc[i,'[제조]총자본순이익률(IFRS연결)'] = df.loc[i,'[제조]총자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'[제조]매출액총이익률(IFRS연결)']):
       df.loc[i,'[제조]매출액총이익률(IFRS연결)'] = df.loc[i,'[제조]매출액총이익률(IFRS)']
    if pd.isnull(df.loc[i,'[제조]자기자본순이익률(IFRS연결)']):
       df.loc[i,'[제조]자기자본순이익률(IFRS연결)'] = df.loc[i,'[제조]자기자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'[제조]총자본정상영업이익률(IFRS연결)']):
       df.loc[i,'[제조]총자본정상영업이익률(IFRS연결)'] = df.loc[i,'[제조]총자본정상영업이익률(IFRS)']
    if pd.isnull(df.loc[i,'자산(*)(요약)(IFRS연결)']):
       df.loc[i,'자산(*)(요약)(IFRS연결)'] = df.loc[i,'자산(*)(요약)(IFRS)']
    if pd.isnull(df.loc[i,'자본(*)(요약)(IFRS연결)']):
       df.loc[i,'자본(*)(요약)(IFRS연결)'] = df.loc[i,'자본(*)(요약)(IFRS)']
    if pd.isnull(df.loc[i,'현금비율(IFRS연결)(백만원)']):
       df.loc[i,'현금비율(IFRS연결)(백만원)'] = df.loc[i,'현금비율(IFRS)']
    if pd.isnull(df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'] = df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'차입금의존도(IFRS연결)']):
       df.loc[i,'차입금의존도(IFRS연결)'] = df.loc[i,'차입금의존도(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'감가상각비(IFRS연결)(천원)']):
       df.loc[i,'감가상각비(IFRS연결)(천원)'] = df.loc[i,'감가상각비(IFRS)(천원)']
    if pd.isnull(df.loc[i,'부채비율(IFRS연결)']):
       df.loc[i,'부채비율(IFRS연결)'] = df.loc[i,'부채비율(IFRS)']
    if pd.isnull(df.loc[i,'당좌비율(IFRS연결)']):
       df.loc[i,'당좌비율(IFRS연결)'] = df.loc[i,'당좌비율(IFRS)']
    if pd.isnull(df.loc[i,'*이자비용(요약)(IFRS연결)(백만원)']):
       df.loc[i,'*이자비용(요약)(IFRS연결)(백만원)'] = df.loc[i,'*이자비용(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'순운전자본비율(IFRS연결)']):
       df.loc[i,'순운전자본비율(IFRS연결)'] = df.loc[i,'순운전자본비율(IFRS)']
    if pd.isnull(df.loc[i, '[제조]      유동성장기부채(요약)(IFRS연결)']):
       df.loc[i, '[제조]      유동성장기부채(요약)(IFRS연결)'] = df.loc[i,'[제조]      유동성장기부채(요약)(IFRS)']
    if pd.isnull(df.loc[i,'[제조]      단기차입금(요약)(IFRS연결)']):
       df.loc[i,'[제조]      단기차입금(요약)(IFRS연결)'] = df.loc[i,'[제조]      단기차입금(요약)(IFRS)']
    if pd.isnull(df.loc[i, '유형자산의 증가(*)(IFRS연결)(천원)']):
       df.loc[i, '유형자산의 증가(*)(IFRS연결)(천원)'] = df.loc[i,'유형자산의 증가(*)(IFRS)(천원)']


In [59]:
drop_col = ['유보율(IFRS)' ,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)','[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)',
'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)','자산(*)(요약)(IFRS)(백만원)', 
'부채(*)(요약)(IFRS)(백만원)','[제조]자본(*)(요약)(IFRS)(백만원)','당기순이익(손실)(요약)(IFRS)(백만원)', 
'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)','금융원가(요약)(IFRS)(백만원)','금융수익(요약)(IFRS)(백만원)',
'매출액(수익)(요약)(IFRS)(백만원)','연구개발비(IFRS)(천원)',
'[제조]      유형자산(요약)(IFRS)(백만원)','자본금회전률(IFRS)',
'총자본회전률(IFRS)','재고자산회전률(IFRS)','유형자산회전율(IFRS)','자기자본회전률(IFRS)','타인자본회전률(IFRS)',
'[제조]경영자본순이익률(IFRS)','[제조]금융비용부담률(IFRS)','[제조]총자본순이익률(IFRS)','[제조]매출액총이익률(IFRS)',
'[제조]자기자본순이익률(IFRS)','[제조]총자본정상영업이익률(IFRS)','자산(*)(요약)(IFRS)','자본(*)(요약)(IFRS)',
 '현금비율(IFRS)','* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)',
 '차입금의존도(IFRS)(백만원)','감가상각비(IFRS)(천원)','부채비율(IFRS)',
'당좌비율(IFRS)','*이자비용(요약)(IFRS)(백만원)','순운전자본비율(IFRS)',
'[제조]자산(*)(요약)(IFRS)(백만원)_x','[제조]자산(*)(요약)(IFRS연결)(백만원)_x',
'[제조]자산(*)(요약)(IFRS)(백만원)_y','[제조]자산(*)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_y', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_x', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_x','유형자산의 증가(*)(IFRS)(천원)',
'[제조]      유동성장기부채(요약)(IFRS)','[제조]      단기차입금(요약)(IFRS)','자산(*)(요약)(IFRS연결)(백만원)','[제조]자본(*)(요약)(IFRS연결)(백만원)']

In [60]:
df.drop(drop_col,axis=1,inplace=True)

In [61]:
## 성장성 지표인 증가율 컬럼 만들기
성장성_col = ['[제조]      유형자산(요약)(IFRS연결)(백만원)','매출액(수익)(요약)(IFRS연결)(백만원)',
           '자본(*)(요약)(IFRS연결)','자산(*)(요약)(IFRS연결)',
           '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']

In [62]:
## 증가율 컬럼 전기에 음수일때 당기에 음수지만 증가할때 증가율 계산
## ex (2011년 -20, 2012년 -10) -> -10-(-20) / np.abs(-20)= 
증가율_df = (df.groupby(['회사명', '거래소코드'])[성장성_col].shift(0) - df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1)) *100
증가율_df.rename(columns={'[제조]      유형자산(요약)(IFRS연결)(백만원)':'유형자산증가율', '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액증가율',
       '자본(*)(요약)(IFRS연결)':'자기자본증가율', '자산(*)(요약)(IFRS연결)':'총자본증가율',
       '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'정상영업이익증가율'},inplace=True)

df.drop(['[제조]      유형자산(요약)(IFRS연결)(백만원)','[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'],axis=1,inplace=True)
df = pd.concat([df,증가율_df],axis=1)

In [63]:
## 시가총액 merge
시가총액['회계년도'] = 시가총액['회계년도'].astype(str).str[:4].astype(int)
df = df.merge(시가총액,on=['거래소코드','회계년도'],how='left')
df['시가총액(백만원)'] = df['시가총액'] / 1000000
df.drop(['시가총액'],axis=1,inplace=True)

## 거시변수 merge
거시변수.rename(columns={'회계연도':'회계년도'},inplace=True)
df = df.merge(거시변수,on=['회계년도'],how='left')

In [64]:
## 기업별 업력 설립일 이후 지속된 기간
df['업력'] = df['회계년도'] -  pd.to_datetime(df['설립일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)
df.drop('설립일',axis=1,inplace=True)

## 누적수익성비율
df['누적수익성비율'] = df['[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']/df['자산(*)(요약)(IFRS연결)']

## 이자부담율
df['이자부담률'] = df['EBITDA(IFRS)(백만원)'] / df['*이자비용(요약)(IFRS연결)(백만원)'] 

## 부채상환계수
df['부채상환계수'] = df['EBITDA(IFRS)(백만원)'] / (df['*이자비용(요약)(IFRS연결)(백만원)']+df['[제조]      단기차입금(요약)(IFRS연결)'] + df['[제조]      유동성장기부채(요약)(IFRS연결)'])

## 현금흐름_자산 = 영업현금흐름 / 전기총자산
df['현금흐름 대 자산'] = df.groupby(['회사명','거래소코드'])['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'].shift(0) / df.groupby(['회사명','거래소코드'])['자산(*)(요약)(IFRS연결)'].shift(0)

## 현금흐름_매출액
df['현금흐름 대 매출액'] = df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] / df['매출액(수익)(요약)(IFRS연결)(백만원)']

# 토빈Q
df['토빈Q'] = (df['부채(*)(요약)(IFRS연결)(백만원)'] + df['시가총액(백만원)']) / df['자산(*)(요약)(IFRS연결)']

## TMD
df['TMD'] = df['부채(*)(요약)(IFRS연결)(백만원)'] / df['시가총액(백만원)']

## 알트만 변수
df['시총/총자산'] = df['시가총액(백만원)'] / df['자산(*)(요약)(IFRS연결)']

## RD -단위도 수정
df['연구개발비(IFRS연결)(백만원)'] = df['연구개발비(IFRS연결)(천원)']/1000
df.drop(['연구개발비(IFRS연결)(천원)'],axis=1,inplace=True)
df['RD'] = df['연구개발비(IFRS연결)(백만원)'] / df['매출액(수익)(요약)(IFRS연결)(백만원)']

## 직원수변동률 , 직원인건비총액(백만원단위)
df['직원수변동률%'] = df.groupby(['회사명','거래소코드'])['합계_기말인원(명)'].pct_change()*100
df['직원인건비총액(백만원)'] = (df['합계_년간총급여(천원)']/1000) * df['합계_기말인원(명)']
df.drop(['합계_년간총급여(천원)'],axis=1,inplace=True)

## 알트만 변수  EBIT = 법인세비용차감전순이익+금융원가-금융수익
df['EBIT/총자산'] = (df['법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)'] + df['금융원가(요약)(IFRS연결)(백만원)']-df['금융수익(요약)(IFRS연결)(백만원)'])/df['자산(*)(요약)(IFRS연결)']

## PBR , PER
df['PBR'] = df['시가총액(백만원)']  / df['자본(*)(요약)(IFRS연결)']

## (전기영업손익 + 당기영업손익)/2
df['당기전기영업손익'] =(df.groupby(['회사명','거래소코드'])['* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'].shift(0) +df.groupby(['회사명','거래소코드'])['* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'].shift(1))/2

## CAPEX
df['유형자산의 증가(*)(IFRS연결)(천원)'] = df['유형자산의 증가(*)(IFRS연결)(천원)']/1000

##FINDEF = (CAPEX - CFR[영업현금흐름]) 
df['FINDEP'] = (df['유형자산의 증가(*)(IFRS연결)(천원)']-df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'])

##
df['EBITDA마진율'] = df['EBITDA(IFRS)(백만원)']/df['매출액(수익)(요약)(IFRS연결)(백만원)']*100

In [65]:
df['PBR변화율']  = (df.groupby(['회사명', '거래소코드'])['PBR'].shift(0) - df.groupby(['회사명', '거래소코드'])['PBR'].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])['PBR'].shift(1)) *100
df['ROA변화율']  = (df.groupby(['회사명', '거래소코드'])['[제조]총자본순이익률(IFRS연결)'].shift(0) - df.groupby(['회사명', '거래소코드'])['[제조]총자본순이익률(IFRS연결)'].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])['[제조]총자본순이익률(IFRS연결)'].shift(1)) *100



In [66]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 7973 개
전체 기업 수 792


In [67]:
rename_list = {'자산(*)(요약)(IFRS연결)':'자산', '자본(*)(요약)(IFRS연결)':'자본',
       '차입금의존도(IFRS연결)':'차입금의존도', '유보율(IFRS연결)(백만원)':'유보율', '현금비율(IFRS연결)(백만원)':'현금비율',
       '부채비율(IFRS연결)':'부채비율', '당좌비율(IFRS연결)':'당좌비율', '순운전자본비율(IFRS연결)':'순운전자본비율',
       '*이자비용(요약)(IFRS연결)(백만원)':'이자비용', '* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'영업손익',
       '감가상각비(IFRS연결)(천원)':'감가상각비', 'EBITDA(IFRS)(백만원)':'EBITDA', '[제조]총자본정상영업이익률(IFRS연결)':'총자본정상영업이익률',
       '[제조]자기자본순이익률(IFRS연결)':'자기자본순이익률', '[제조]총자본순이익률(IFRS연결)':'총자본순이익률', '[제조]매출액총이익률(IFRS연결)':'매출액총이익률',
       '[제조]금융비용부담률(IFRS연결)':'금융비용부담률', '[제조]경영자본순이익률(IFRS연결)':'경영자본순이익률', '자본금회전률(IFRS연결)':'자본금회전률',
       '유형자산회전율(IFRS연결)':'유형자산회전율', '재고자산회전률(IFRS연결)':'재고자산회전률', '자기자본회전률(IFRS연결)':'자기자본회전률',
       '타인자본회전률(IFRS연결)':'타인자본회전률', '총자본회전률(IFRS연결)':'총자본회전률', 
       '[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)':'이익잉여금',
       '[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)':'영업활동으로 인한 현금흐름',
       '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액', '금융원가(요약)(IFRS연결)(백만원)':'금융원가',
       '금융수익(요약)(IFRS연결)(백만원)':'금융수익', '법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)':'법인세비용차감전순이익',
       '당기순이익(손실)(요약)(IFRS연결)(백만원)':'당기순이익', 
       '부채(*)(요약)(IFRS연결)(백만원)':'부채', '외국인_주식분포비율':'외국인지분율', '보통주식비율':'대주주지분율',
       '[제조]      단기차입금(요약)(IFRS연결)':'단기차입금', '[제조]      유동성장기부채(요약)(IFRS연결)':'유동성장기부채',
       '시가총액(백만원)':'시가총액','연구개발비(IFRS연결)(백만원)':'연구개발비','직원인건비총액(백만원)':'직원인건비총액',
       '통계청 한국표준산업분류 10차(대분류)':'산업군','유형자산의 증가(*)(IFRS연결)(천원)':'CAPEX'}

df.rename(columns=rename_list,inplace=True)

In [68]:
df.isna().sum()

회사명             0
거래소코드           0
회계년도            0
자산            145
자본            145
             ... 
당기전기영업손익      937
FINDEP        145
EBITDA마진율     213
PBR변화율       1206
ROA변화율       1000
Length: 71, dtype: int64

In [69]:
df['상장일'] = pd.to_datetime(df['상장일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)

In [70]:
# ## 회계년도 2011년 제거
df = df[df['회계년도']>2011]

print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 7249 개
전체 기업 수 781


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7249 entries, 1 to 7972
Data columns (total 71 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   회사명             7249 non-null   object 
 1   거래소코드           7249 non-null   object 
 2   회계년도            7249 non-null   int64  
 3   자산              7136 non-null   float64
 4   자본              7136 non-null   float64
 5   차입금의존도          7074 non-null   float64
 6   유보율             7074 non-null   float64
 7   현금비율            7075 non-null   float64
 8   부채비율            7074 non-null   float64
 9   당좌비율            7074 non-null   float64
 10  순운전자본비율         7074 non-null   float64
 11  이자비용            7136 non-null   float64
 12  영업손익            7136 non-null   float64
 13  감가상각비           7136 non-null   float64
 14  EBITDA          7074 non-null   float64
 15  산업군             7249 non-null   object 
 16  총자본정상영업이익률      7074 non-null   float64
 17  자기자본순이익률        7074 non-null   f

-----

### Y 부실 붙이기

-----

In [72]:
부실 = pd.read_csv('./datasets/y/y값최종.csv',index_col=0)

In [73]:
부실

,번호,회사명,거래소코드,회계년도,공시제목
625,35,디패션,30420,2010,상장폐지기업
633,27,지앤알,43630,2010,상장폐지기업
632,28,유니텍전자,39040,2010,상장폐지기업
631,29,세븐코스프,17160,2010,상장폐지기업
630,30,맥스브로,88810,2010,상장폐지기업
...,...,...,...,...,...
28,632,티티씨디펜스,309900,2021,상장폐지기업
30,630,에스에이치엔엘,50320,2021,상장폐지기업
37,623,현진소재,53660,2021,상장폐지기업
38,622,스포츠서울,39670,2021,상장폐지기업


In [74]:
부실['회계년도'] = 부실['회계년도'].astype(int)
부실['거래소코드'] = 부실['거래소코드'].map(padding)
부실.drop(['번호','회사명'],axis=1,inplace=True)

In [75]:
df = df.merge(부실,on=['회계년도','거래소코드'],how='left')

In [76]:
부실['공시제목'].value_counts()

상장폐지기업      301
회생절차개시결정     60
Name: 공시제목, dtype: int64

In [77]:
부실.head()

,거래소코드,회계년도,공시제목
625,030420,2010,상장폐지기업
633,043630,2010,상장폐지기업
632,039040,2010,상장폐지기업
631,017160,2010,상장폐지기업
630,088810,2010,상장폐지기업


In [78]:
## 상폐기업 라벨링

df["공시제목"].fillna(0,inplace=True)

상폐_ind = df[df["공시제목"]== '상장폐지기업'].index

### set1
df.loc[상폐_ind,"target_1"] = 1

for i in df.index:
    if (df.loc[i,"target_1"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) :
        df.loc[i-1, "target_1"] = 1
df['target_1'].fillna(0,inplace=True)
df['target_1'] = df['target_1'].astype(int)

# ### set2
df.loc[상폐_ind,"target_2"] = 1

for i in df.index:
    if (df.loc[i,"target_2"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) and (df.loc[i,'공시제목']=='상장폐지기업') :
        df.loc[i-1, "target_2"] = 1 
    if (df.loc[i,"target_2"] == 1) and (df.loc[i,"회사명"] == df.loc[i-2,"회사명"]) and (df.loc[i,'공시제목']=='상장폐지기업'):
        df.loc[i-2, "target_2"] = 1

df['target_2'].fillna(0,inplace=True)
df['target_2'] = df['target_2'].astype(int)

## set_3

df.loc[상폐_ind,"target_3"] = 1

for i in df.index:
    if (df.loc[i,"target_3"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) and (df.loc[i,'공시제목']=='상장폐지기업'):
        df.loc[i-1, "target_3"] = 1
df['target_3'].fillna(0,inplace=True)
df['target_3'] = df['target_3'].astype(int)



In [79]:
print(df.target_1.sum(),df.target_2.sum(),df.target_3.sum())

33 44 33


In [80]:
## 회생기업 라벨링

df["공시제목"].fillna(0,inplace=True)

회생_ind = df[df["공시제목"]== '회생절차개시결정'].index

### set1
df.loc[회생_ind,"target_1"] = 1

for i in df.index:
    if (df.loc[i,"target_1"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) & (df.loc[i,'공시제목']=='회생절차개시결정'):
        df.loc[i-1, "target_1"] = 1
df['target_1'].fillna(0,inplace=True)
df['target_1'] = df['target_1'].astype(int)

# ### set2
df.loc[회생_ind,"target_2"] = 1

df['target_2'].fillna(0,inplace=True)
df['target_2'] = df['target_2'].astype(int)

## set_3

df.loc[회생_ind,"target_3"] = 1

df['target_3'].fillna(0,inplace=True)
df['target_3'] = df['target_3'].astype(int)



In [81]:
print(df.target_1.sum(),df.target_2.sum(),df.target_3.sum())

61 59 48


In [82]:
def get_index_with_50_or_more_nan_columns(df):
    """
    데이터프레임에서 결측치인 컬럼의 개수가 50개 이상인 인덱스 번호를 출력하는 함수
    :param df: 데이터프레임
    :return: 결측치 컬럼 개수가 50개 이상인 인덱스 번호 리스트
    """
    # 결측치 컬럼 개수 계산
    nan_column_counts = df.isnull().sum(axis=1)
    
    # 결측치 컬럼 개수가 50개 이상인 인덱스 번호 추출
    index_with_50_or_more_nan_columns = nan_column_counts[nan_column_counts >= 50].index.to_list()
    print('제거행 수:' , len(index_with_50_or_more_nan_columns))

    df.drop(index=index_with_50_or_more_nan_columns,inplace=True)
    df.reset_index(drop=True,inplace=True)
    return df


In [83]:
get_index_with_50_or_more_nan_columns(df)

제거행 수: 113


,회사명,거래소코드,회계년도,자산,자본,차입금의존도,유보율,현금비율,부채비율,당좌비율,...,PBR,당기전기영업손익,FINDEP,EBITDA마진율,PBR변화율,ROA변화율,공시제목,target_1,target_2,target_3
0,(주)DB하이텍,000990,2012,1157755.0,290893.0,58.88,66.82,7.19,298.00,41.39,...,0.965470,-29386.0,-44923.428,16.298089,-4.104438,64.953887,0,0,0,0
1,(주)DB하이텍,000990,2013,1058392.0,204757.0,62.73,28.12,7.39,416.90,31.58,...,1.631640,-12616.5,7911.723,15.821712,68.999640,-181.578947,0,0,0,0
2,(주)DB하이텍,000990,2014,984877.0,120703.0,66.85,-10.15,10.72,715.95,47.39,...,1.696375,17974.5,-17540.415,22.410512,3.967457,-0.667557,0,0,0,0
3,(주)DB하이텍,000990,2015,984644.0,254057.0,53.39,24.76,44.31,287.57,104.20,...,2.497314,85264.5,-57154.084,31.072813,47.214718,270.689655,0,0,0,0
4,(주)DB하이텍,000990,2016,986789.0,341917.0,42.80,64.24,42.15,188.60,106.69,...,2.063216,148698.5,-156057.027,32.477902,-17.382593,-30.380730,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7131,흥아해운(주),003280,2017,848959.0,116884.0,31.84,87.23,6.17,626.32,29.58,...,0.820581,-3601.5,53377.346,1.021459,15.622930,-314.795918,0,0,0,0
7132,흥아해운(주),003280,2018,803698.0,73148.0,31.60,-13.66,4.89,998.73,28.66,...,0.965285,-25347.5,45031.927,-2.201602,17.634325,-28.782288,0,0,0,0
7133,흥아해운(주),003280,2019,430963.0,30919.0,72.50,-42.42,6.75,1293.83,42.55,...,1.744108,-24980.0,12387.180,6.009083,80.683190,20.534862,0,0,0,0
7134,흥아해운(주),003280,2020,286776.0,-26284.0,87.97,-142.71,3.21,0.00,33.73,...,-1.145830,-9081.0,4129.588,10.802887,-165.697221,-70.432692,0,0,0,0


In [84]:
print(df.target_1.sum(),df.target_2.sum(),df.target_3.sum())

61 59 48


In [85]:
print('부실기업데이터 수:',df[df['target_1']==1].shape[0])
print('부실기업데이터 수:',df[df['target_2']==1].shape[0])
print('부실기업데이터 수:',df[df['target_3']==1].shape[0])
print('전체데이터수:',df.shape[0],'개')

부실기업데이터 수: 61
부실기업데이터 수: 59
부실기업데이터 수: 48
전체데이터수: 7136 개


In [86]:
print('결측치존재하는 기업 수 : ',len(df[df.isna().any(1)]['회사명'].unique()),'개')

결측치존재하는 기업 수 :  166 개


In [87]:
pd.reset_option('display.max_rows')
# pd.set_option('display.max_rows',None)
df.isna().sum()

회사명           0
거래소코드         0
회계년도          0
자산            0
자본            0
           ... 
ROA변화율      163
공시제목          0
target_1      0
target_2      0
target_3      0
Length: 75, dtype: int64

In [88]:
## 결측치가 62개 존재하는 모든 컬럼 제거
## 첫 상장 년도와 회계년도 같은 년도 
 
df = df[df['회계년도']>df['상장일']].reset_index(drop=True)
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 6704 개
전체 기업 수 757


In [89]:
print('결측치존재하는 기업 수 : ',len(df[df.isna().any(1)]['회사명'].unique()),'개')

결측치존재하는 기업 수 :  66 개


In [90]:
# 한국콜마 2013년 데이터  결측치확인
# 2012년 데이터가 존재하지 않음 증가율 
# -> 한국 콜마데이터 2013년 삭제
print(df[df['매출액증가율'].isna()])
df.drop(index=6182,axis=0,inplace=True)

print('결측치존재하는 기업 수 : ',len(df[df.isna().any(1)]['회사명'].unique()),'개')

          회사명   거래소코드  회계년도        자산       자본  차입금의존도  유보율  현금비율  부채비율  당좌비율  \
6182  한국콜마(주)  161890  2013  230014.0  97807.0     NaN  NaN  6.07   NaN   NaN   

      ...       PBR  당기전기영업손익   FINDEP  EBITDA마진율  PBR변화율 ROA변화율  공시제목  \
6182  ...  5.438332       NaN -8270.05        NaN     NaN    NaN     0   

      target_1  target_2  target_3  
6182         0         0         0  

[1 rows x 75 columns]
결측치존재하는 기업 수 :  66 개


In [91]:
## 직원수변동률
## 2011년 회사상장후 임원으로 만 구성
## 본격적으로 직원은 2012년 부터 존재 따라서 직원수변동률 0으로 처리
df[df['직원수변동률%'].isna()][['회사명','회계년도','직원수변동률%','직원인건비총액']]
df.loc[714,'직원수변동률%']=0

In [92]:
## 직원수변동률
## 한화화인케미칼 2015년 한화케미칼에 합병 

print(df[df['직원인건비총액'].isna()][['회사명','회계년도','직원수변동률%','직원인건비총액','target_1']])
df.drop(index=6479,axis=0,inplace=True)

             회사명  회계년도  직원수변동률%  직원인건비총액  target_1
6479  한화화인케미칼(주)  2015      0.0      NaN         0


In [93]:
## 대주주지분율 외국인지분율 결측치인 데이터
df[df['대주주지분율'].isna()][['회사명','회계년도','대주주지분율']]

## 미래산업(주) 2013,2014,2015,년 대주주지분율 0으로 대체
## (주)풍산 2013년 대주주지분율 33.35로 dart에서 확인
## 동양건설산업 2013년  1.81로 dart에서 확인

df.loc[614,'대주주지분율']=1.81
df.loc[2513,'대주주지분율']=33.35
df.loc[3983,'대주주지분율']=0
df.loc[3984,'대주주지분율']=0
df.loc[3985,'대주주지분율']=0

In [94]:
df[df['외국인지분율'].isna()][['회사명','회계년도','외국인지분율','target_1']]

,회사명,회계년도,외국인지분율,target_1
280,(주)넥솔론,2016,NaN,1
357,(주)다함이텍,2012,NaN,1
592,(주)동성화학,2020,NaN,0
1238,(주)신일건업,2014,NaN,1
1246,(주)신한,2019,NaN,1
1357,(주)알앤엘바이오,2012,NaN,1
1388,(주)에스와이코퍼레이션,2012,NaN,0
2463,(주)티이씨앤코,2015,NaN,0
2501,(주)포스코플랜텍,2015,NaN,1
2772,(주)한화갤러리아타임월드,2019,NaN,0


In [95]:
## 외국인 지분율 결측치는 실제 0인 값
## 그이유는 해당 년도 근처에서 관리종목 또는 횡령 또는 상폐를 앞두고있음

df['외국인지분율'].fillna(0,inplace=True)

In [96]:
pd.set_option('display.max_columns',None)
df.describe()

,회계년도,자산,자본,차입금의존도,유보율,현금비율,부채비율,당좌비율,순운전자본비율,이자비용,영업손익,감가상각비,EBITDA,총자본정상영업이익률,자기자본순이익률,총자본순이익률,매출액총이익률,금융비용부담률,경영자본순이익률,자본금회전률,유형자산회전율,재고자산회전률,자기자본회전률,타인자본회전률,총자본회전률,이익잉여금,영업활동으로 인한 현금흐름,매출액,금융원가,금융수익,법인세비용차감전순이익,당기순이익,부채,합계_기말인원(명),외국인지분율,대주주지분율,단기차입금,유동성장기부채,상장일,CAPEX,유형자산증가율,매출액증가율,자기자본증가율,총자본증가율,정상영업이익증가율,시가총액,장단기금리차,cpi,업력,누적수익성비율,이자부담률,부채상환계수,현금흐름 대 자산,현금흐름 대 매출액,토빈Q,TMD,시총/총자산,연구개발비,RD,직원수변동률%,직원인건비총액,EBIT/총자산,PBR,당기전기영업손익,FINDEP,EBITDA마진율,PBR변화율,ROA변화율,target_1,target_2,target_3
count,6702.000000,6.702000e+03,6.702000e+03,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6.702000e+03,6.702000e+03,6.702000e+03,6.702000e+03,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6.702000e+03,6.702000e+03,6.702000e+03,6.702000e+03,6.702000e+03,6.702000e+03,6.702000e+03,6.702000e+03,6702.000000,6702.000000,6702.000000,6.702000e+03,6.702000e+03,6702.00000,6.702000e+03,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6.702000e+03,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6702.000000,6.702000e+03,6702.000000,6702.000000,6.702000e+03,6702.000000,6702.000000,6.702000e+03,6.702000e+03,6702.000000,6702.000000,6.658000e+03,6702.000000,6702.000000,6702.000000
mean,2016.608923,3.908229e+06,1.796135e+06,23.792563,2561.009132,39.130909,167.536679,154.545495,12.555561,3.074171e+04,1.872361e+05,2.718017e+07,2.065906e+05,3.346006,-1.157754,1.619417,19.012347,1.060049,1.993784,47.037250,9.388400,275.101802,2.255291,2.431218,0.942741,1.273933e+06,2.706687e+05,2.901526e+06,8.054340e+04,6.029131e+04,1.721353e+05,1.304027e+05,2.112095e+06,1727.350343,10.013967,44.141973,2.334999e+05,1.668183e+05,1993.25291,2.028870e+05,38.652572,18.185299,12.047845,7.176376,38.440450,1.809334e+06,1.061186,96.959434,39.860340,0.248374,NaN,NaN,0.044803,0.046364,1.142951,2.113782,0.669293,4.906975e+04,0.009122,3.744529,3.448109e+09,0.028740,1.283267,1.809761e+05,-6.778169e+04,5.041048,13.727882,-inf,0.009102,0.008803,0.007162
std,2.879306,1.811070e+07,1.000348e+07,17.640616,7660.664133,102.964071,1181.095602,233.392656,21.803509,1.175945e+05,1.640646e+06,1.925603e+08,1.644696e+06,6.780955,54.375982,10.432607,17.907457,3.961801,14.990056,200.411343,73.611544,10740.284709,2.286017,1.944893,0.521855,9.326403e+06,2.219719e+06,1.196881e+07,4.484624e+05,4.367639e+05,1.729436e+06,1.319448e+06,9.781548e+06,5945.861086,13.064872,16.773249,8.812667e+05,7.600740e+05,13.84060,1.440907e+06,1142.489704,588.864506,194.083380,40.209934,1810.033582,1.270773e+07,0.681532,3.274313,19.464681,0.431619,NaN,NaN,0.075220,0.450366,0.831925,4.247373,0.864939,6.711755e+05,0.021522,62.885307,4.816504e+10,0.101166,2.309022,1.533666e+06,9.932287e+05,18.791575,166.660224,NaN,0.094975,0.093419,0.084332
min,2012.000000,7.110000e+03,-2.666885e+06,0.000000,-2347.490000,0.000000,0.000000,3.970000,-149.650000,0.000000e+00,-5.860143e+06,0.000000e+00,-3.435112e+06,-72.880000,-2195.070000,-141.530000,-480.680000,-164.820000,-255.810000,0.100000,0.010000,0.000000,0.000000,0.010000,0.000000,-4.477062e+06,-3.480396e+06,2.234000e+03,0.000000e+00,0.000000e+00,-7.085214e+06,-5.229225e+06,5.650000e+02,2.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,1956.00000,0.000000e+00,-99.962978,-99.686359,-1229.612403,-96.210010,-38730.434783,5.572724e+03,0.210000,91.815000,1.000000,-5.744726,-inf,-inf,-0.752662,-30.937151,0.273894,0.008365,0.010280,0.000000e+00,0.000000,-99.714286,0.000000e+00,-1.202215,-72.114413,-2.417180e+06,-3.747546e+07,-504.472799,-4523.160314,-inf,0.000000,0.000000,0.000000
25%,2014.000000,2.098532e+05,1.068468e+05,8.292500,302.902500,8.452500,43.670000,64.860000,-1.540000,7.900000e+02,2.431500e+03,3.532505e+05,5.115582e+03,0.920000,-0.247500,-0.140000,9.350000,-0.070000,-0.180000,7.570000,1.710000,5.150000,1.070000,1.300000,0.610000,3.195375e+04,2.464000e+03,1.724730e+05,1.509000e+03,8.300000

In [97]:
df = df[~(df['이자부담률'].isin([np.inf,-np.inf]))]

In [98]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 6493 개
전체 기업 수 754


In [99]:
df[(df['ROA변화율'].isin([np.inf,-np.inf]))]

,회사명,거래소코드,회계년도,자산,자본,차입금의존도,유보율,현금비율,부채비율,당좌비율,순운전자본비율,이자비용,영업손익,감가상각비,EBITDA,산업군,총자본정상영업이익률,자기자본순이익률,총자본순이익률,매출액총이익률,금융비용부담률,경영자본순이익률,자본금회전률,유형자산회전율,재고자산회전률,자기자본회전률,타인자본회전률,총자본회전률,이익잉여금,영업활동으로 인한 현금흐름,매출액,금융원가,금융수익,법인세비용차감전순이익,당기순이익,부채,합계_기말인원(명),외국인지분율,대주주지분율,단기차입금,유동성장기부채,상장일,CAPEX,유형자산증가율,매출액증가율,자기자본증가율,총자본증가율,정상영업이익증가율,시가총액,장단기금리차,cpi,업력,누적수익성비율,이자부담률,부채상환계수,현금흐름 대 자산,현금흐름 대 매출액,토빈Q,TMD,시총/총자산,연구개발비,RD,직원수변동률%,직원인건비총액,EBIT/총자산,PBR,당기전기영업손익,FINDEP,EBITDA마진율,PBR변화율,ROA변화율,공시제목,target_1,target_2,target_3
1954,(주)이원,017940,2016,3544179.0,1183228.0,46.93,3204.73,11.44,199.53,89.33,6.14,46097.0,11088.0,38393826.0,93510.55,도매 및 소매업,0.31,-2.35,-1.08,10.34,0.78,-1.55,116.50,4.08,12.35,3.30,1.72,1.13,1056264.0,28158.0,3995901.0,47891.0,15011.0,-33821.0,-37940.0,2360952.0,256.0,4.86,45.33,414679.0,259419.0,1997,45677.943,-5.29861,-14.891556,-4.270276,0.980550,-68.926380,430122.0000,1.25,95.783,32,0.298028,2.028560,0.129841,0.007945,0.007047,0.787509,5.489029,0.121360,0.000,0.00000,-6.227106,5.485568e+06,-0.000266,0.363516,23385.5,17519.943,2.340162,6.672473,-inf,0,0,0,0
3956,무림페이퍼(주),009200,2016,1904532.0,591501.0,56.89,269.93,14.75,221.98,40.87,-15.28,32361.0,42062.0,562331.0,50633.12,제조업,2.06,0.06,-0.07,12.89,2.76,-0.08,10.88,1.01,5.45,1.91,0.78,0.56,105726.0,200756.0,1131454.0,62976.0,27467.0,-977.0,-1438.0,1313031.0,395.0,5.63,54.10,432527.0,243362.0,1990,18965.886,-4.65988,-2.188677,-0.776176,-12.244103,-41.915349,110264.6715,1.25,95.783,43,0.055513,1.564634,0.071490,0.105410,0.177432,0.747320,11.907994,0.057896,2512.374,0.00222,-22.549020,9.679870e+06,0.018131,0.186415,57238.5,-181790.114,4.475049,-12.291312,-inf,0,0,0,0
5412,전방(주),000950,2014,506373.0,176669.0,52.98,2181.64,1.90,186.62,16.01,-34.21,10271.0,-11304.0,172007.0,3892.19,제조업,-2.19,-12.87,-4.69,0.05,4.14,-5.38,29.19,0.72,3.78,1.30,0.75,0.48,60208.0,-1914.0,245206.0,10271.0,113.0,-24517.0,-24182.0,329703.0,815.0,0.28,16.39,243275.0,15130.0,1968,5663.140,-3.44694,-16.552320,-11.841376,-3.632253,-254.595186,57792.0000,1.50,94.196,61,0.118900,0.378949,0.014487,-0.003780,-0.007806,0.765236,5.704994,0.114129,0.000,0.00000,-6.857143,1.644162e+07,-0.028357,0.327120,-1996.0,7577.140,1.587314,39.110777,-inf,0,0,0,0


In [100]:
df = df[~(df['ROA변화율'].isin([np.inf,-np.inf]))]

In [101]:
## 금융업제거
df = df[~df['산업군'].str.contains('금융')]
df = df[~(df['산업군'].str.contains('부동산'))]  # 부동산업 3개기업 18개행 제거

In [102]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 6415 개
전체 기업 수 744


In [103]:

df['현금흐름/총부채비율'] = df['영업활동으로 인한 현금흐름'] / df['부채']
df['영업현금흐름-단기차입금'] = df['영업활동으로 인한 현금흐름'] - df['단기차입금']
df['abs(영업현금흐름-당기순이익)/매출액'] = (np.abs(df['영업활동으로 인한 현금흐름'] - df['당기순이익']))/df['매출액']


df.drop(['자산','이자비용','매출액','부채','금융원가','시가총액','EBITDA','공시제목','상장일'],axis=1,inplace=True)

In [104]:
pd.reset_option('display.max_rows')
## inf값 존재  - 이자비용=0일때 나옴
# 이자부담률  = EBITDA / 이자비용 
# 부채상환계수  = EBITDA / (이자비용+단기차입금+유동성장기부채)  
df[df.isin([np.inf,-np.inf]).any(1)]['target_1'].value_counts() ## inf 행 제거

Series([], Name: target_1, dtype: int64)

In [105]:
df[df.isin([np.inf,-np.inf]).any(1)]

,회사명,거래소코드,회계년도,자본,차입금의존도,유보율,현금비율,부채비율,당좌비율,순운전자본비율,영업손익,감가상각비,산업군,총자본정상영업이익률,자기자본순이익률,총자본순이익률,매출액총이익률,금융비용부담률,경영자본순이익률,자본금회전률,유형자산회전율,재고자산회전률,자기자본회전률,타인자본회전률,총자본회전률,이익잉여금,영업활동으로 인한 현금흐름,금융수익,법인세비용차감전순이익,당기순이익,합계_기말인원(명),외국인지분율,대주주지분율,단기차입금,유동성장기부채,CAPEX,유형자산증가율,매출액증가율,자기자본증가율,총자본증가율,정상영업이익증가율,장단기금리차,cpi,업력,누적수익성비율,이자부담률,부채상환계수,현금흐름 대 자산,현금흐름 대 매출액,토빈Q,TMD,시총/총자산,연구개발비,RD,직원수변동률%,직원인건비총액,EBIT/총자산,PBR,당기전기영업손익,FINDEP,EBITDA마진율,PBR변화율,ROA변화율,target_1,target_2,target_3,현금흐름/총부채비율,영업현금흐름-단기차입금,abs(영업현금흐름-당기순이익)/매출액


In [106]:
df.describe()

,회계년도,자본,차입금의존도,유보율,현금비율,부채비율,당좌비율,순운전자본비율,영업손익,감가상각비,총자본정상영업이익률,자기자본순이익률,총자본순이익률,매출액총이익률,금융비용부담률,경영자본순이익률,자본금회전률,유형자산회전율,재고자산회전률,자기자본회전률,타인자본회전률,총자본회전률,이익잉여금,영업활동으로 인한 현금흐름,금융수익,법인세비용차감전순이익,당기순이익,합계_기말인원(명),외국인지분율,대주주지분율,단기차입금,유동성장기부채,CAPEX,유형자산증가율,매출액증가율,자기자본증가율,총자본증가율,정상영업이익증가율,장단기금리차,cpi,업력,누적수익성비율,이자부담률,부채상환계수,현금흐름 대 자산,현금흐름 대 매출액,토빈Q,TMD,시총/총자산,연구개발비,RD,직원수변동률%,직원인건비총액,EBIT/총자산,PBR,당기전기영업손익,FINDEP,EBITDA마진율,PBR변화율,ROA변화율,target_1,target_2,target_3,현금흐름/총부채비율,영업현금흐름-단기차입금,abs(영업현금흐름-당기순이익)/매출액
count,6415.000000,6.415000e+03,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6.415000e+03,6.415000e+03,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6.415000e+03,6.415000e+03,6.415000e+03,6.415000e+03,6.415000e+03,6415.000000,6415.000000,6415.000000,6.415000e+03,6.415000e+03,6.415000e+03,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6415.000000,6.415000e+03,6415.000000,6415.000000,6.415000e+03,6415.000000,6415.000000,6.415000e+03,6.415000e+03,6415.000000,6415.000000,6373.000000,6415.000000,6415.000000,6415.000000,6415.000000,6.415000e+03,6415.000000
mean,2016.642401,1.851129e+06,24.397975,2538.257189,35.195989,171.709330,141.390666,11.646137,1.932216e+05,2.823326e+07,3.258530,-1.481763,1.451718,18.836535,1.129584,1.764705,47.318042,8.492465,285.738027,2.283428,2.347043,0.944404,1.308423e+06,2.804314e+05,6.234064e+04,1.776482e+05,1.333200e+05,1761.238504,9.861930,44.121772,2.378333e+05,1.707634e+05,2.099155e+05,39.089966,18.278255,12.252678,7.245104,39.635255,1.056691,96.995703,39.998753,0.236672,214.411929,152.612165,0.044540,0.045211,1.133105,2.161794,0.651762,5.102413e+04,0.009085,3.653413,3.589259e+09,0.027605,1.269700,1.864996e+05,-7.051590e+04,4.846265,14.034616,-14.919466,0.009197,0.008885,0.007171,0.134955,4.259807e+04,0.111414
std,2.887228,1.021958e+07,17.405463,7775.603487,86.828647,1206.800695,185.433437,21.094773,1.676390e+06,1.967504e+08,6.750686,55.122356,10.289552,17.934704,3.984015,14.821910,204.494332,40.318980,10977.802914,2.301663,1.840396,0.518295,9.528865e+06,2.267884e+06,4.461705e+05,1.767255e+06,1.346941e+06,6059.895523,12.928254,16.816960,8.937795e+05,7.746288e+05,1.472147e+06,1165.721764,601.315015,198.241298,40.792955,1843.180561,0.679018,3.282379,19.474488,0.422457,3870.656526,3751.470985,0.074228,0.446736,0.805995,4.314259,0.836867,6.859500e+05,0.021297,60.026791,4.922538e+10,0.100852,2.310802,1.567108e+06,1.014423e+06,17.132428,170.103548,1521.608171,0.095467,0.093850,0.084382,0.261192,1.881936e+06,0.689369
min,2012.000000,-2.666885e+06,0.000000,-2347.490000,0.000000,0.000000,3.970000,-149.650000,-5.860143e+06,0.000000e+00,-72.880000,-2195.070000,-141.530000,-480.680000,-164.820000,-255.810000,0.100000,0.010000,0.000000,0.000000,0.010000,0.000000,-4.477062e+06,-3.480396e+06,0.000000e+00,-7.085214e+06,-5.229225e+06,2.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-99.962978,-99.686359,-1229.612403,-96.210010,-38730.434783,0.210000,91.815000,1.000000,-5.744726,-3616.650000,-3282.390000,-0.752662,-30.937151,0.273894,0.009032,0.010280,0.000000e+00,0.000000,-99.714286,0.000000e+00,-1.202215,-72.114413,-2.417180e+06,-3.747546e+07,-504.472799,-4523.160314,-45700.000000,0.000000,0.000000,0.000000,-3.807515,-1.715492e+07,0.000006
25%,2014.000000,1.076870e+05,9.610000,292.490000,8.275000,46.720000,64.080000,-1.865000,2.465500e+03,3.697220e+05,0.910000,-0.465000,-0.255000,9.290000,-0.025000,-0.315000,7.525000,1.690000,5.150000,1.090000,1.290000,0.620000,3.167950e+04,2.631000e+03,8.480000e+02,1.080000e+02,-9.115000e+02,182.000000,1.480000,32.090000,9.122000e+03,0.000000e+00,3.462110e+03,-3.799121,-5.487670,-0.922408,-2.255547,-35.467164,0.510000,94.196000,27.000000,0.095694,1.295869,0.076298,0.009946,0.011760,0.770777,0.444432,0.235965,0.000000e+0

In [107]:
print('최종기업수:',len(df['회사명'].unique()))
df.reset_index(drop=True,inplace=True)

최종기업수: 744


In [108]:
df.shape

(6415, 69)

---------------------------------------------------------------------------

In [130]:
df['target_1'].value_counts()

0    6172
1      59
Name: target_1, dtype: int64

In [110]:
#부실정의 컬럼
df['sum'] = df['target_1']+df['target_2']+df['target_3']

## 임시 부실데이터저장
temp = pd.DataFrame()
for i in df['회사명'].unique():
    a = df[(df['회사명']==i) & (df['sum']!=0)]
    temp = pd.concat([temp,a],axis=0)
    
temp.drop(columns='sum',axis=1,inplace=True)

for i in temp['회사명'].unique():
    df = df[~(df['회사명']==i)]


In [111]:
df = pd.concat([temp,df],axis=0)
df.drop(columns=['sum'],axis=1,inplace=True)


In [112]:
df['target_3'].value_counts()

0    6227
1      46
Name: target_3, dtype: int64

In [113]:
df['target_2'].value_counts()

0    6216
1      57
Name: target_2, dtype: int64

In [114]:
print('부실기업데이터 수:',df[df['target_1']==1].shape[0])
print('부실기업데이터 수:',df[df['target_2']==1].shape[0])
print('부실기업데이터 수:',df[df['target_3']==1].shape[0])
print('전체데이터수',df.shape[0],'개')

부실기업데이터 수: 59
부실기업데이터 수: 57
부실기업데이터 수: 46
전체데이터수 6273 개


In [115]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 6273 개
전체 기업 수 744


In [116]:
df.target_2.value_counts()

0    6216
1      57
Name: target_2, dtype: int64

In [117]:
df[df['target_2']==1]['산업군'].value_counts()

건설업                         25
제조업                         23
운수 및 창고업                     4
전문, 과학 및 기술 서비스업             2
정보통신업                        2
사업시설 관리, 사업 지원 및 임대 서비스업     1
Name: 산업군, dtype: int64

In [118]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '자본', '차입금의존도', '유보율', '현금비율', '부채비율', '당좌비율',
       '순운전자본비율', '영업손익', '감가상각비', '산업군', '총자본정상영업이익률', '자기자본순이익률', '총자본순이익률',
       '매출액총이익률', '금융비용부담률', '경영자본순이익률', '자본금회전률', '유형자산회전율', '재고자산회전률',
       '자기자본회전률', '타인자본회전률', '총자본회전률', '이익잉여금', '영업활동으로 인한 현금흐름', '금융수익',
       '법인세비용차감전순이익', '당기순이익', '합계_기말인원(명)', '외국인지분율', '대주주지분율', '단기차입금',
       '유동성장기부채', 'CAPEX', '유형자산증가율', '매출액증가율', '자기자본증가율', '총자본증가율',
       '정상영업이익증가율', '장단기금리차', 'cpi', '업력', '누적수익성비율', '이자부담률', '부채상환계수',
       '현금흐름 대 자산', '현금흐름 대 매출액', '토빈Q', 'TMD', '시총/총자산', '연구개발비', 'RD',
       '직원수변동률%', '직원인건비총액', 'EBIT/총자산', 'PBR', '당기전기영업손익', 'FINDEP',
       'EBITDA마진율', 'PBR변화율', 'ROA변화율', 'target_1', 'target_2', 'target_3',
       '현금흐름/총부채비율', '영업현금흐름-단기차입금', 'abs(영업현금흐름-당기순이익)/매출액'],
      dtype='object')

In [119]:
train = df[df['회계년도']<=2018]
test = df[df['회계년도']>=2019]

In [120]:
train.target_1.value_counts()

0    4188
1      50
Name: target_1, dtype: int64

In [121]:
test.target_2.value_counts()

0    2029
1       6
Name: target_2, dtype: int64

In [122]:
test[test['target_2']==1]['산업군'].value_counts()

제조업      4
건설업      1
정보통신업    1
Name: 산업군, dtype: int64

In [123]:
df = df[~(df.isna().any(1))]

In [124]:
df.describe()

,회계년도,자본,차입금의존도,유보율,현금비율,부채비율,당좌비율,순운전자본비율,영업손익,감가상각비,총자본정상영업이익률,자기자본순이익률,총자본순이익률,매출액총이익률,금융비용부담률,경영자본순이익률,자본금회전률,유형자산회전율,재고자산회전률,자기자본회전률,타인자본회전률,총자본회전률,이익잉여금,영업활동으로 인한 현금흐름,금융수익,법인세비용차감전순이익,당기순이익,합계_기말인원(명),외국인지분율,대주주지분율,단기차입금,유동성장기부채,CAPEX,유형자산증가율,매출액증가율,자기자본증가율,총자본증가율,정상영업이익증가율,장단기금리차,cpi,업력,누적수익성비율,이자부담률,부채상환계수,현금흐름 대 자산,현금흐름 대 매출액,토빈Q,TMD,시총/총자산,연구개발비,RD,직원수변동률%,직원인건비총액,EBIT/총자산,PBR,당기전기영업손익,FINDEP,EBITDA마진율,PBR변화율,ROA변화율,target_1,target_2,target_3,현금흐름/총부채비율,영업현금흐름-단기차입금,abs(영업현금흐름-당기순이익)/매출액
count,6231.000000,6.231000e+03,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6.231000e+03,6.231000e+03,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6.231000e+03,6.231000e+03,6.231000e+03,6.231000e+03,6.231000e+03,6231.000000,6231.000000,6231.000000,6.231000e+03,6.231000e+03,6.231000e+03,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6.231000e+03,6231.000000,6231.000000,6.231000e+03,6231.000000,6231.000000,6.231000e+03,6.231000e+03,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6231.000000,6.231000e+03,6231.000000
mean,2016.649976,1.890637e+06,24.332733,2577.159907,35.364455,169.059342,142.075399,11.784290,1.978702e+05,2.889069e+07,3.362062,-1.246455,1.557596,19.025197,1.030859,1.940327,47.836879,8.334818,290.746888,2.277514,2.369581,0.947973,1.347633e+06,2.870538e+05,6.357846e+04,1.819295e+05,1.365804e+05,1785.203338,9.903832,44.218949,2.418743e+05,1.730649e+05,2.145042e+05,36.249642,17.536868,11.966162,7.220624,43.544333,1.056498,97.002995,40.201252,0.249874,220.218677,157.023588,0.045493,0.049000,1.130061,2.133441,0.651668,5.238556e+04,0.009226,3.632144,3.689432e+09,0.028447,1.269045,1.913398e+05,-7.254958e+04,5.034510,13.389062,-7.563236,0.009469,0.009148,0.007382,0.138282,4.517947e+04,0.106594
std,2.887722,1.036511e+07,17.320197,7866.102401,87.800439,1222.236692,187.398396,21.064352,1.700563e+06,1.995489e+08,6.692111,53.149195,9.975589,17.953209,3.632612,14.068234,207.138658,39.964966,11138.344957,2.304950,1.851342,0.516860,9.665584e+06,2.300645e+06,4.524990e+05,1.792720e+06,1.366268e+06,6140.553594,12.793989,16.666097,9.050787e+05,7.842079e+05,1.493310e+06,1165.379625,609.722814,200.622783,40.819326,1862.168186,0.678311,3.282479,19.365777,0.405679,3927.237583,3806.374718,0.073291,0.439873,0.809591,4.266853,0.839865,6.959541e+05,0.021488,60.440232,4.994333e+10,0.096557,2.110015,1.589695e+06,1.029016e+06,16.274152,166.709628,1519.616849,0.096854,0.095213,0.085610,0.261464,1.908948e+06,0.693468
min,2012.000000,-2.666885e+06,0.000000,-2347.490000,0.000000,0.000000,3.970000,-149.650000,-5.860143e+06,0.000000e+00,-72.880000,-2195.070000,-141.530000,-480.680000,-164.820000,-255.810000,0.120000,0.010000,0.000000,0.000000,0.010000,0.000000,-4.477062e+06,-3.480396e+06,0.000000e+00,-7.085214e+06,-5.229225e+06,2.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-99.962978,-99.686359,-1229.612403,-96.210010,-38730.434783,0.210000,91.815000,2.000000,-5.744726,-3616.650000,-3282.390000,-0.752662,-30.937151,0.273894,0.009032,0.010280,0.000000e+00,0.000000,-99.714286,0.000000e+00,-1.202215,-66.336144,-2.417180e+06,-3.747546e+07,-504.472799,-4523.160314,-45700.000000,0.000000,0.000000,0.000000,-3.807515,-1.715492e+07,0.000006
25%,2014.000000,1.095200e+05,9.615000,307.710000,8.295000,46.015000,64.155000,-1.745000,2.856000e+03,3.802440e+05,1.005000,-0.150000,-0.095000,9.420000,-0.030000,-0.115000,7.810000,1.690000,5.165000,1.100000,1.310000,0.620000,3.556000e+04,2.897500e+03,8.485000e+02,3.290000e+02,-3.190000e+02,181.000000,1.520000,32.460000,9.500000e+03,0.000000e+00,3.591280e+03,-3.723240,-5.353784,-0.866165,-2.125228,-35.389245,0.510000,94.196000,27.000000,0.107849,1.369813,0.079597,0.011292,0.012772,0.767832,0.442531,0.236813,0.000000e+00,

In [125]:
df['target_1'].value_counts()

0    6172
1      59
Name: target_1, dtype: int64

In [126]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 6231 개
전체 기업 수 741


In [127]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '자본', '차입금의존도', '유보율', '현금비율', '부채비율', '당좌비율',
       '순운전자본비율', '영업손익', '감가상각비', '산업군', '총자본정상영업이익률', '자기자본순이익률', '총자본순이익률',
       '매출액총이익률', '금융비용부담률', '경영자본순이익률', '자본금회전률', '유형자산회전율', '재고자산회전률',
       '자기자본회전률', '타인자본회전률', '총자본회전률', '이익잉여금', '영업활동으로 인한 현금흐름', '금융수익',
       '법인세비용차감전순이익', '당기순이익', '합계_기말인원(명)', '외국인지분율', '대주주지분율', '단기차입금',
       '유동성장기부채', 'CAPEX', '유형자산증가율', '매출액증가율', '자기자본증가율', '총자본증가율',
       '정상영업이익증가율', '장단기금리차', 'cpi', '업력', '누적수익성비율', '이자부담률', '부채상환계수',
       '현금흐름 대 자산', '현금흐름 대 매출액', '토빈Q', 'TMD', '시총/총자산', '연구개발비', 'RD',
       '직원수변동률%', '직원인건비총액', 'EBIT/총자산', 'PBR', '당기전기영업손익', 'FINDEP',
       'EBITDA마진율', 'PBR변화율', 'ROA변화율', 'target_1', 'target_2', 'target_3',
       '현금흐름/총부채비율', '영업현금흐름-단기차입금', 'abs(영업현금흐름-당기순이익)/매출액'],
      dtype='object')

In [132]:
df.to_csv('./datasets/전처리준비/코스피결측치처리완료.csv',encoding='utf-8-sig')